##
**Diagnosing Patients with Dementia**

You are a data scientist working for a healthcare organisation that aims to improve early detection of Dementia. The organisation
has provided you with a rich dataset containing health information for 2,149 patients. Each patient is uniquely identified by a
Patient ID, and the dataset contains a variety of features such as demographic details, lifestyle factors, medical history, clinical
measurements, cognitive assessments, and symptoms.

Your task is to build a machine learning model to predict whether a patient has Dementia based on the available data. This
classification model will help doctors identify high-risk patients and prioritise them for further diagnostic tests or interventions.

The dataset is named "dementia.csv" and can be downloaded from the “Project Datasets” folder on myLMS.

Appendix A has a detailed description of all the columns within the dataset.

1.1.
Performing the necessary preprocessing on the data to get it ready for training the machine learning model. This will
include removing missing values, encoding categorical variables, feature standardisation and any necessary feature engineering.
(5 Marks)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

categorical_cols = df.select_dtypes(include=['object', 'category']).columns


#load data
path = 'dementia.csv'

data = pd.read_csv(path)

#clean
df_cleaned = data.dropna()
edu = df_cleaned['EducationLevel']

for col in categorical_cols:
    df_cleaned[f'{col}_encoded'] = le.fit_transform(df_cleaned[col])

BoolCols = df.select_dtypes(include=['bool']).columns
df_encoded = pd.get_dummies(df_cleaned, columns=categorical_cols, drop_first=True) # drop_first avoids multicollinearity

'''#encoding categorical variables
categorical_cols = df_cleaned.select_dtypes(include=[ 'EducationLevel']).columns

le = LabelEncoder()
for col in categorical_cols:
    df_cleaned[col + '_encoded'] = le.fit_transform(df_cleaned[col])
'''



#print(data.head())
#print(df_cleaned.head())
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
print(f"Categorical columns identified: {list(categorical_cols)}")
'''
Categorical columns identified: 
['Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'Forgetfulness', 'DoctorInCharge', 'Age_group']
'''
#print(categorical_cols)

/tmp/ipykernel_31146/1603819789.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[f'{col}_encoded'] = le.fit_transform(df_cleaned[col])


KeyError: "['Age_group'] not in index"

1.2.
Implement both of the following approaches for feature selection:

a) A filter-based method using Select KBest

b) A wrapper-based method using Recursive Feature Elimination (RFE) with a suitable estimator

Your implementation should clearly show the selected features in each case.
(10 marks)

1.3.
For both RFE and SelectKBest:

-Discuss the advantages and limitations of the methods
(5 marks)

-Explain the process for selecting the optimal number of features for each approach 
(5 marks)